In [10]:
debug = False
kaggle = False
resize = False

In [11]:
import csv
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import json
import numpy as np
import random
import os

In [12]:
tf.__version__
seed = 18
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(random)
tf.keras.backend.set_floatx('float64')

ntasks = 5
channels = 18
filters = 3
perceptionsize = 3
neurons = 128
epochs = 200
iterations = 25

In [13]:
ids = {}
names = []

#with os.scandir('/kaggle/input/abstraction-and-reasoning-challenge/') as entries:
#    for entry in entries:
#        names.append("/kaggle/input/abstraction-and-reasoning-challenge/"+entry.name)
if kaggle:
    path = "/kaggle/input/abstraction-and-reasoning-challenge/"
else:
    path = "./"

with os.scandir(path+'test/') as entries:
    for entry in entries:
        cid = entry.name.replace(".json", "")
        names.append(path+"test/"+entry.name)
        ids[path+"test/"+entry.name] = cid

In [34]:
def load(name):
    X = []
    testcases = []
    Y = []
    lastw = 0
    lasth = 0
    with open(name) as json_file:
        data = json.load(json_file) 
        l = len(data['train'])
        i = 0
        for t in data['train']:
            i += 1
            
            cin = np.array(t['input'])
            cout = np.array(t['output'])

            if not resize:
                if cin.shape != cout.shape:
                    return False
            elif (lastw != cin.shape[0] or lasth != cin.shape[1]) and lastw == 0:
                    return False
            else:
                    lastw = cin.shape[0]
                    lasth = cin.shape[1]
            
            if resize:
                cX = np.zeros((30, 30, channels))
                cY = np.zeros((30, 30, channels))
            
                cinr = np.full((30, 30), -1)
                cinr[:cin.shape[0], :cin.shape[1]] = cin
                cX[:, :, 0] = cinr
                    #! Clone?
                cX[:, :, 1] = cinr
            
                coutr = np.full((30, 30), -1)
                coutr[:cout.shape[0], :cout.shape[1]] = cout
                cY[:, :, 0] = coutr
                #! Clone?
                cY[:, :, 1] = coutr
            else:
                cX = np.zeros((cin.shape[0], cin.shape[1], channels))
                cY = np.zeros((cin.shape[0], cin.shape[1], channels))
                
                cX[:, :, 0] = cin
                cX[:, :, 1] = cin
                
                cY[:, :, 0] = cout
                cY[:, :, 1] = cout

                
            X.append(cX)
            Y.append(cY)
            
        for t in data['test']:
            i += 1
            
            cin = np.array(t['input'])

            if resize:
                cX = np.zeros((30, 30, channels))
            
                cinr = np.full((30, 30), -1)
                cinr[:cin.shape[0], :cin.shape[1]] = cin
                cX[:, :, 0] = cinr
                    #! Clone?
                cX[:, :, 1] = cinr
            
            else:
                cX = np.zeros((cin.shape[0], cin.shape[1], channels))
                
                cX[:, :, 0] = cin
                cX[:, :, 1] = cin
            
            testcases.append(cX)

    return [np.array(X), np.array(Y), np.array(testcases)]

In [35]:
for name in names:
    cin = load(name)
    if cin == False:
        print("SIZE DIFF: ERR")
    else:
        print(name, len(cin[2]))

./test/3ed85e70.json 1
./test/05a7bcf2.json 1
./test/20981f0e.json 1
./test/0d87d2a6.json 1
./test/1c56ad9f.json 1
./test/2a5f8217.json 1
./test/31adaf00.json 1
./test/2c737e39.json 1
./test/15113be4.json 1
SIZE DIFF: ERR
SIZE DIFF: ERR
SIZE DIFF: ERR
SIZE DIFF: ERR
SIZE DIFF: ERR
./test/3391f8c0.json 1
./test/009d5c81.json 1
SIZE DIFF: ERR
./test/1e81d6f9.json 1
./test/0a2355a6.json 1
./test/27a77e38.json 1
./test/33b52de3.json 1
./test/08573cc6.json 1
./test/2685904e.json 1
./test/18419cfa.json 1
SIZE DIFF: ERR
./test/0607ce86.json 1
./test/256b0a75.json 1
./test/0f63c0b9.json 1
SIZE DIFF: ERR
./test/1d0a4b61.json 1
./test/00dbd492.json 1
./test/40f6cd08.json 1
SIZE DIFF: ERR
./test/12eac192.json 1
SIZE DIFF: ERR
./test/423a55dc.json 1
./test/292dd178.json 1
SIZE DIFF: ERR
SIZE DIFF: ERR
SIZE DIFF: ERR
./test/070dd51e.json 1
./test/1c02dbbe.json 1
./test/137f0df0.json 1
./test/21f83797.json 1
SIZE DIFF: ERR
SIZE DIFF: ERR
./test/1acc24af.json 1
SIZE DIFF: ERR
./test/2546ccf6.json 1
S

In [16]:
import matplotlib.pyplot as plt
if debug:
    def plot(inp):
        plt.imshow(inp[:, :, 0].reshape(30, 30))
        plt.colorbar()
        plt.show()

    def plotf(inp):
        plt.imshow(inp)
        plt.colorbar()
        plt.show()

In [17]:
def loss_f(x, y):
    return tf.reduce_mean(tf.square(x[:, :, :, 0]-y[:, :, :, 0]))

def train(model, ctask, epochs, iterations, channels, verbose=False, validate=False, wholetask=None):
    trainer = tf.keras.optimizers.Adam()

    origx = ctask[0]
    x = origx.copy()
    y = ctask[1]

    mask = np.zeros((x.shape[0], x.shape[1], x.shape[2], channels))
    mask[:, :, :, 1] = 1
    xl2 = origx*mask

    for k in range(epochs):
        x = origx.copy()
        with tf.GradientTape() as g:
            for i in tf.range(iterations):
                dx = model(x)
                x = x+(dx*1)
                x = x*(1-mask)+xl2
            loss = loss_f(x, y)
            grads = g.gradient(loss, model.weights)
            #grads = [g/(tf.norm(g)+1e-8) for g in grads]
            trainer.apply_gradients(zip(grads, model.weights))

    return model

def generate(model, cin, channels, iterations):
    mask = np.zeros((cin.shape[0], cin.shape[1], cin.shape[2], channels))
    mask[:, :, :, 1] = 1
    xl2 = cin*mask
    
    for i in range(iterations):
        dx = model(cin)
        cin = cin+(dx*1)
        cin = cin*(1-mask)+xl2
        if i%10 == 0:
            pass
            #toprint = cin.numpy()[0]
            #plot(toprint)
    
    toprint = cin.numpy()
    return toprint

In [32]:
ntasks = 5
channels = 18
filters = 3
perceptionsize = 3
neurons = 128
epochs = 200
iterations = 25

def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    str_pred = str_pred.replace('-1', '0')
    return str_pred

def solve(taskname):
    models = []
    modelcount = 1

    ctask = load(taskname)
    if ctask == False:
        return False

    cin = [ctask[0], ctask[1]]
    ctest = ctask[2]
    
    if debug:
        plot(ctask[0][0])
        plot(ctask[1][0])
        plot(ctest[0])

    outs = []

    for i in range(modelcount):
        print("MODEL "+str(i))
        print(cin[0].shape)
        model = tf.keras.Sequential([
            tf.keras.layers.DepthwiseConv2D((perceptionsize, perceptionsize), input_shape=(cin[0].shape[1], cin[0].shape[2], channels), strides=[1, 1], padding="same", depth_multiplier=filters, activation="relu"),
            tf.keras.layers.Conv2D(neurons, 1, activation="relu"),
            tf.keras.layers.Conv2D(channels, 1, activation=None, kernel_initializer=tf.zeros_initializer)
        ])
        model = train(model, cin, epochs, iterations, channels, False, False)
        out = generate(model, ctest, channels, iterations)
        outs.append(out)

    outputs = []
    out = np.array(outs)
    print(out.shape)
    fout = np.clip(np.round(np.mean(out, axis=0)), -1, 9)
    print(fout.shape)
    for task in fout:
        if debug:
            print(task.shape)
            plot(task)
        final = task[:, :, 0]
        x = np.argmin(final[0, :])
        y = np.argmin(final[:, 0])
        cropped = final[:y, :x]
        finalout = flattener(cropped.astype(int).tolist())
        if debug:
            plotf(cropped)
            print(finalout)
        outputs.append(finalout)
    return outputs

In [36]:
with open('submission.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile, delimiter=',')
    writer.writerow(['output_id','output'])
    with open(path+'sample_submission.csv') as infile:
        reader = csv.reader(infile, delimiter=',')
        lastid = 0
        crow = 0
        for row in reader:
            cid = row[0]
            if cid != "output_id":
                crow += 1
                taskid, testno = cid.split("_")
                if lastid == taskid:
                    print("SAME SKIP")
                    continue
                lastid = taskid
                name = path+'test/'+str(taskid)+'.json'
                solutions = solve(name)
                if solutions == False:
                    print("NA")
                    writer.writerow([cid, '|111|111|111|'])
                    lastid = 0
                else:
                    i = 0
                    for s in solutions:
                        if s != '[]':
                            cname = ids[name]
                            print(cname+"_"+str(i), s)
                            writer.writerow([cname+"_"+str(i), s])
                        else:
                            cname = ids[name]
                            print(cname+"_"+str(i), s)
                            writer.writerow([cname+"_"+str(i), '|111|111|111|'])
                            print("FAILED TO SOLVE")
                        i += 1
                outfile.flush()
                print("FINISHED LINE MAYBE "+str(crow))

NA
FINISHED LINE MAYBE 1
MODEL 0
(5, 14, 14, 18)
(1, 1, 14, 14, 18)
(1, 14, 14, 18)
009d5c81_0 []
FAILED TO SOLVE
FINISHED LINE MAYBE 2
MODEL 0
(4,)


IndexError: tuple index out of range